## Prepare dataset for SOMS using BIOPERIANT12 CNCLNG01 1995-2009
24/10/22 <br>
Includes the missing files <br>
Circularity index

## Eddy Detection with Okubo-Weiss
Use ndimage to group together cyclonic and anticyclonic eddies similar to the eddy detection algorithm 

Instead of using ssh can use our OW masks 

Run through same criteria 

### List changes
* Dates from the original code
* load_lonlat: changed the pathroot 
* load_eta:
    * changed pathroot**
    * changed eta to sossheig (instead of interpolating - there is an error at 0°E and interpolation doens't need to be done, but need to tell it where to slice using find_ind
* Change spherical law of cosines in distance matrix:
    * due to rounding errors in np.radians it gives an error ~0.2km in the case I was seeing
    * rounding st the 8th decimal produces this error
    * write my own radians2 formula to do it properly
* time_counter = 0 2007m04d01 gridT so need to remove it from okubo and from RV

In [1]:
import sys
import numpy as np
import scipy as sp
import netCDF4 as nc4
import matplotlib.pyplot as plt
import xarray as xr
import dask.array as da
import pandas as pd
#from glob import glob
import glob

import numpy.linalg as linalg
import scipy.signal as signal
import scipy.ndimage as ndimage
import scipy.interpolate as interpolate

from netCDF4 import Dataset

from itertools import repeat

import sys
sys.path.insert(0, '/home/tsmith/scratch/eddyTracking/ecjoliver/')
import params 

import re

from scipy.io.netcdf import netcdf_file as netcdf
import cmocean.cm as cmo
import warnings
import matplotlib.colors as colors
from cartopy import crs as ccrs, feature as cfeature

In [26]:
from numba import jit, njit

In [2]:
# #Ask dask for 8 workers to distribute processing load
# from dask.distributed import Client, progress
# client = Client(threads_per_worker=1, n_workers=8)
# client

In [3]:
%matplotlib inline

In [4]:
def find_ind(grid1d, coord):
        a=abs(grid1d-coord)
        return np.where(a==np.min(a))[0][0]

In [5]:
#dask? but dask is only xarray

## Import relative vorticity

In [6]:
file_listRV = glob.glob('/home/tsmith/scratch/Okubo/BP12-CNCLNG01/depth/BIOPERIANT12-CNCLNG01_y200*m**d**_RV.nc')
file_listRV.sort()

In [7]:
grid_RV = xr.open_mfdataset(file_listRV, decode_times=False,engine='netcdf4')

In [8]:
lats = grid_RV['lat'][0]
lons = grid_RV['lon'][0]

In [9]:
j1 = find_ind(lons[0,:],-11)
j2 = find_ind(lons[0,:],15)
i1 = find_ind(lats[:,0],-60)
i2 = find_ind(lats[:,0],-37)

In [10]:
RV = grid_RV['Relative_vorticity'][:,0,i1:i2+1,j1:j2+1]

In [11]:
#to use in circularity
lonsRVx = grid_RV['lon'][0,i1:i2+1,j1:j2+1]
latsRVx = grid_RV['lat'][0,i1:i2+1,j1:j2+1]

[589:1016] is -60°S to -37°S

[3313:3626] is -11° to 15°E

## Import okubo mask files

In [12]:
file_listO=glob.glob("/home/tsmith/scratch/Okubo/BP12-CNCLNG01/depth/BIOPERIANT12-CNCLNG01_y200*[5-9]m**d**_Okubomask.nc")
file_listO.sort()


In [13]:
gridO = xr.open_mfdataset(file_listO, decode_times=False,engine='netcdf4')

In [14]:
Okubo_mask = gridO['okubo_mask'][:,i1:i2+1,j1:j2+1]

In [15]:
RV_mask = RV.where(Okubo_mask == 1)

In [16]:
#make nan=0
RV_mask = RV_mask.fillna(0)

In [17]:
%%time
rv_mask = RV_mask.values

CPU times: user 1min 54s, sys: 19min 49s, total: 21min 44s
Wall time: 25min 54s


In [18]:
#going to try go through without .values and keep as xarray
#but will probably have to change it

In [19]:
#rv_mask = RV_mask.values

In [20]:
# lon_all = grid['lon'][0,0,:]
# lat_all = grid['lat'][0,:,0]

In [21]:
def raw_nemo_globber_specifytpe(exp_path,return_dates=False):
    """function that globs for NEMO files from the raw experiment output looking for velocity files only.

    Parameters
    ----------
    exp_path:  path to NEMO experiment (will glob through the year directories)
    1) 'grid_T_2D' : glob all files of type grid_T_2D

    return_dates:  optional. will run find_dates function before returning and so will return a pandas DataFrame

    :returns: 

    For 'grid_T_2D' : single list of globbed files

    Will return pandas DataFrame instead if return_dates=True

    WARNING: if return_dates is true you will get one line for every new date in the pandas dataframe!
    (There will be multiple references to the same file.)

    Notes
    -------
    Runs reg_date by default.

    Typical NEMO output looks like:
    
        cordex24-ERAI01_1d_19910101_19911231_grid_T_2D.nc

    Example 
    -------

    """
    #_lg.info("We are globbing for experiment type: " + os.path.basename(exp_path))
    #_lg.info("We are globbing for the following file type: " + file_type)
    glob_pattern='*_gridT.nc'
    #glob_pattern='*/*_grid_T_2D.nc'

    infiles=sorted(glob.glob(exp_path + glob_pattern ))
    assert(infiles!=[]),"glob didn't find anything!"

    #if isinstance(date_extraction,(list,tuple)):
        #infiles=globbed_date_remove(infiles)

    if return_dates:
        return find_dates(infiles)
    return infiles


def reg_date(string_to_find_date_in):
    """function that uses regular expressions to find start and end date from file.
    
    :string_to_find_date_in: string from NEMO netCDF file.
    :returns: string with start and end date
    """
    #really handonline regex finder: https://regex101.com/#python
    exp = re.search(r'[0-9]{4}m[0-9]{2}d[0-9]{2}', string_to_find_date_in) 
    #print(exp)
    exp=exp.group()
    
    #print exp
    return exp

def find_dates(globbedfiles):
    """function that finds dates associated with passed list of NEMO output. Returns in pandas DataFrame.

    NB: assumes one day per NEMO output.
    modified ts

    Parameters
    ----------
    :globbedfiles: list of globbed files from raw_nemo_globber with pattern:
        */cordex24*_1d_*_grid_T_2D.nc

    :returns: dataframe of names and dates in pandas DataFrame.
    """
    time_index = []
    for file in globbedfiles:
        dates_nemo=pd.Timestamp(int(reg_date(file)[0:4]),int(reg_date(file)[5:7]),
                                      int(reg_date(file)[8:10]))
        
        time_index.append(dates_nemo)

    time_index = pd.Series(time_index)
    #might have to change file_time_index to 0 for all - because we have one file for every entry
    #file_time_index = list(range(len(globbedfiles)))
    file_time_index = 0
    file_list = globbedfiles
#     print(time_index)
#     print(file_time_index)
#     print(file_list)
    nemo_df=pd.DataFrame({'date':time_index,\
                          'file_time_index':file_time_index,\
                          'file_list':file_list\
                          })

    nemo_df.index=nemo_df.date
    del nemo_df['date']
    return nemo_df



Will have to re-write eta code for when looping over multiple years

Pathroot

Problem with 2007m04d01 no time counter=0 so dropping it and redoing with T=1094

Also issue with loading 2006m01d01 wrong time counter (so it is overwritten by 2006m01d03 as they are the same), but should be fine if importing files one at a time

In [40]:
def find_nearest(array, value):
    idx=(np.abs(array-value)).argmin()
    return array[idx], idx

def nanmean(array, axis=None):
    return np.mean(np.ma.masked_array(array, np.isnan(array)), axis)

def load_lonlat(run, disk='erebor'):
    '''
    Loads latitude and longitude vectors from OFAM runs or AVISO
    Assumes root is /mnt/erebor unless cerberus is specified.
    '''

    #    
    # OFAM
    #
    if run=='cb_NEMO':
        #just need one file for lon/lat so its fine
        pathroot='/mnt/nrestore/users/ERTH0834/BIOPERIANT12/BIOPERIANT12-CNCLNG01-S/LIN_INTERP/2005/'

        # Find week's map
        #file_header = '*/cordex24-ERAI01_1d_*_grid_T_2D'
        #file_list = glob.glob(pathroot + file_header + '*.nc')
        file_list=raw_nemo_globber_specifytpe(pathroot,return_dates=False)

        assert (len(file_list)>0),"globbing failed, exiting"

        #this is just for the lat/lon info so we only need one time step..

        filename = file_list[0]
        #print filename

        fileobj = Dataset(filename,mode='r')
        #lon = fileobj.variables['nav_lon'][:]
        #lat = fileobj.variables['nav_lat'][:]
        #h'm need to interpolate...

#         lon=np.arange(params.lon1,params.lon2,.25)
#         lat=np.arange(params.lat1,params.lat2,.25)
        
#         lon=np.arange(params.lon1,params.lon2,.083)
#         lat=np.arange(params.lat1,params.lat2,.083)

#read lon/lat straight from NEMO
        lon_all = fileobj.variables['nav_lon'][:]
        lat_all = fileobj.variables['nav_lat'][:]

        lat = lat_all[find_ind(lat_all[:,0],params.lat1):find_ind(lat_all[:,0],params.lat2)+1,0]
        lon = lon_all[0,find_ind(lon_all[0,:],params.lon1):find_ind(lon_all[0,:],params.lon2)+1]
        
        
    return lon, lat


def restrict_lonlat(lon, lat, lon1, lon2, lat1, lat2):
    '''
    Restricts latitude and longitude vectors given
    input limits.
    '''

    tmp, i1 = find_nearest(lon, lon1)
    tmp, i2 = find_nearest(lon, lon2)
    tmp, j1 = find_nearest(lat, lat1)
    tmp, j2 = find_nearest(lat, lat2)

    lon = lon[i1:i2+1]
    lat = lat[j1:j2+1]

    return lon, lat, i1, i2, j1, j2


def load_eta(run, tt, i1, i2, j1, j2, disk='erebor'):
    '''
    Loads sea surface height field from OFAM runs or Aviso
    Assumes root is /mnt/erebor unless cerberus is specified.
     run = 'CTRL' or 'A1B' or 'AVISO'
    '''

    #    
    # OFAM
    #
    if run=='cb_NEMO':
#         def nemo_fixdateline(netcdf_datasetobj):
#             """function to return fixed version of netcdf nav_lon variable
            
#             :netcdf_datasetobj: netCDF4 Dataset object of nemo file
#             :returns: nemo_lons numpy array with fixed dateline
#             """
#             nemo_lons=netcdf_datasetobj.variables['nav_lon'][:]
#             #fix the dateline
#             for index in np.arange(np.shape(nemo_lons)[0]):
#                 start=np.where(np.sign(nemo_lons[index,:])==-1)[0][0]
#                 nemo_lons[index,start:]=nemo_lons[index,start:]+360
#             return nemo_lons

        
        
        #pathroot='/srv/ccrc/data42/z3457920/20151012_eac_sep_dynamics/nemo_cordex24_ERAI01/'
        #pathroot=params.pathroot
        
        #TS: overwriting params.pathroot 
        
        pathroot='/mnt/nrestore/users/ERTH0834/BIOPERIANT12/BIOPERIANT12-CNCLNG01-S/LIN_INTERP/200*[5-9]/'
        
        
        file_list=raw_nemo_globber_specifytpe(pathroot,return_dates=True)
        #TS remove OST problems from gridU,V
        #drop all missing files (other files - gridU,gridV)
#         for i in missing:
            
#             file_list = file_list.drop(i)
        
        infile=file_list.iloc[tt]['file_list']
        file_time_index=file_list.iloc[tt]['file_time_index']

        #h'm need to interpolate because of funky NEMO grid...
        fileobj = Dataset(infile,mode='r')
        #print(fileobj)
        #loni=nemo_fixdateline(fileobj)
        #loni = fileobj.variables['nav_lon'][:]
        
        
#         loni = fileobj.variables['nav_lon'][:]
        
#         lati = fileobj.variables['nav_lat'][:]

#         lon=np.arange(params.lon1,params.lon2,.25)
#         lat=np.arange(params.lat1,params.lat2,.25)
        
#         lon=np.arange(params.lon1,params.lon2,.083)
#         lat=np.arange(params.lat1,params.lat2,.083)

        loni = fileobj.variables['nav_lon'][:]
        lati = fileobj.variables['nav_lat'][:]

        lat = lati[find_ind(lati[:,0],params.lat1):find_ind(lati[:,0],params.lat2)+1,0]
        lon = loni[0,find_ind(loni[0,:],params.lon1):find_ind(loni[0,:],params.lon2)+1]
        #print(lat)
        #print(lon)
        #old_grid_data=fileobj.variables['zos'][file_time_index,:,:]
        old_grid_data=fileobj.variables['sossheig'][file_time_index,:,:]
        XI, YI = np.meshgrid(lon,lat)
        
        #interp
        #TS: don't need to interpolate so take straight from old_grid_data 
        #etamask=interpolate.griddata((loni.flatten(),lati.flatten()),old_grid_data.flatten() , (XI,YI),method='nearest')

        #eta=interpolate.griddata((loni.flatten(),lati.flatten()),old_grid_data.flatten() , (XI,YI),method='cubic')
        
        #TS: my change
        eta = old_grid_data[find_ind(lati[:,0],params.lat1):find_ind(lati[:,0],params.lat2)+1,find_ind(loni[0,:],
                                                                params.lon1):find_ind(loni[0,:],params.lon2)+1]
        
        #set mask
        #TS: won't be any 0
        #eta[np.where(etamask==0)]=0
        eta_miss=[0]

    return eta, eta_miss[0]



def quick_plot(field,findrange=False):
    '''
    Create quick interactive diagnostic plot to double check eddy_detection is doing what we want...
    '''
    y,x=np.meshgrid(np.arange(field.shape[1]),np.arange(field.shape[0]))
    plt.clf()

    if not findrange:
        plt.contourf(y, x, field, levels=np.arange(-2.5,2.5,0.05))
    else:
        if np.isnan(np.sum(field)):
            plotfield=np.nan_to_num(field)
            print('range of field is:')
            print('min',np.min(plotfield))
            print('max',np.max(plotfield))

            plt.contourf(y, x, field,levels=np.linspace(np.min(plotfield),np.max(plotfield),50))

        else:
            print('range of field is:')
            print('min',np.min(field))
            print('max',np.max(field))

            plt.contourf(y, x, field,levels=np.linspace(np.min(field),np.max(field),50))
    plt.title('diagnostic plot')
    plt.show()
    import ipdb; ipdb.set_trace()

def remove_missing(field, missing, replacement):
    '''
    Replaces all instances of 'missing' in 'field' with 'replacement'
    '''

    field[field==missing] = replacement

    return field


def interp_nans(data, indices):
    '''
    Linearly interpolates over missing values (np.nan's) in data
    Data is defined at locations in vector indices.
    '''

    not_nan = np.logical_not(np.isnan(data))

    return np.interp(indices, indices[not_nan], data[not_nan])


def match_missing(data1, data2):
    '''
    Make all locations that are missing in data2 also missing in data1
    Missing values are assumed to be np.nan.
    '''

    data1[np.isnan(data2)] = np.nan
    return data1


def spatial_filter(field, lon, lat, res, cut_lon, cut_lat):
    '''
    Performs a spatial filter, removing all features with
    wavelenth scales larger than cut_lon in longitude and
    cut_lat in latitude from field (defined in grid given
    by lon and lat).  Field has spatial resolution of res
    and land identified by np.nan's
    '''
    #adding res_lat because of non-uniform lat grid
    #it is the average spacing of the grid from -60 to -37
    res_lat = 0.053870249539613724
    
    field_filt = np.zeros(field.shape)

    # see Chelton et al, Prog. Ocean., 2011 for explanation of factor of 1/5
    sig_lon = (cut_lon/5.) / res
    sig_lat = (cut_lat/5.) / res_lat

    land = np.isnan(field)
    field[land] = nanmean(field)

    field_filt = field - ndimage.gaussian_filter(field, [sig_lat, sig_lon])

    field_filt[land] = np.nan

    return field_filt

@njit
def radians2(coord):
    '''TS: Problem with np.radians rounding off the value
    very sensitive to decimal place values up to 9/10
    so using this instead and hoping it works better'''
    rad = coord*np.pi/180
    return rad

@njit
def distance_matrix(lons,lats):
    '''Calculates the distances (in km) between any two cities based on the formulas
    c = sin(lati1)*sin(lati2)+cos(longi1-longi2)*cos(lati1)*cos(lati2)
    d = EARTH_RADIUS*Arccos(c)
    where EARTH_RADIUS is in km and the angles are in radians.
    https://gis.stackexchange.com/questions/4906/
    why-is-law-of-cosines-more-preferable-than-haversine-when-calculating-distance-b/
    9197#comment11102_4906 
    TS: made changes np.radians converted to radians2'''

    EARTH_RADIUS = 6378.1
    X = len(lons)
    Y = len(lats)
    assert X == Y, 'lons and lats must have same number of elements'

    d = np.zeros((X,X))

    #Populate the matrix.
    #spherical law of cosines = haversine (mathematically)
    for i2 in range(len(lons)):
        lati2 = lats[i2]
        loni2 = lons[i2]
        c = np.sin(radians2(lats)) * np.sin(radians2(lati2)) + \
            np.cos(radians2(lons-loni2)) * \
            np.cos(radians2(lats)) * np.cos(radians2(lati2))
        
        #to avoid rounding errors which can happen so leave them as 0
        d[c<1,i2] = EARTH_RADIUS * np.arccos(c[c<1])

    return d


In [38]:
@njit
def distance_sloc(lat1,lat2,lon1,lon2):
    EARTH_RADIUS = 6378.1
    c = np.sin(radians2(lat1)) * np.sin(radians2(lat2)) + \
        np.cos(radians2(lon1-lon2)) * \
        np.cos(radians2(lat1)) * np.cos(radians2(lat2))
    
    d = EARTH_RADIUS * np.arccos(c)
    return(d)

In [70]:
#@njit
def detect_eddies(relvort, field, lon, lat, res, Npix_min, Npix_max, amp_thresh, d_thresh, cyc='anticyclonic'):
    len_deg_lat = 111.325 # length of 1 degree of latitude [km]

    llon, llat = np.meshgrid(lon, lat)

    lon_eddies = np.array([])
    lat_eddies = np.array([])
    amp_eddies = np.array([])
    area_eddies = np.array([])
    scale_eddies = np.array([])
    circ_eddies = np.array([])

    # 1. Find all regions with relvort greater (less than) 0 for anticyclonic (cyclonic) eddies 
    if cyc == 'anticyclonic':
        regions, nregions = ndimage.label( (relvort > 0).astype(int) )
    elif cyc == 'cyclonic':
        regions, nregions = ndimage.label( (relvort < 0).astype(int) )

    for iregion in range(nregions):
        #print("iregion: ", iregion)
   
    # 2. Calculate number of pixels comprising detected region, reject if not within [Npix_min, Npix_max]
        region = (regions==iregion+1).astype(int)
        region_Npix = region.sum()
        #print("Npix: ", region_Npix)
        eddy_area_within_limits = (region_Npix < Npix_max) * (region_Npix > Npix_min)
        #print(eddy_area_within_limits)



    # 3. Detect presence of local maximum (minimum) for anticylonic (cyclonic) eddies, reject if non-existent
        interior = ndimage.binary_erosion(region)
        #exterior = region.astype(bool) - interior
        #problem with exterior line above won't allow bool - bool so kept as 1/0
        exterior = region - interior.astype(int)
        #need to change exterior to T/F instead of 1 and 0
        exterior = exterior.astype(bool)

        if interior.sum() == 0:
            continue
        if cyc == 'anticyclonic':
            has_internal_ext = field[interior].max() > field[exterior].max()
        elif cyc == 'cyclonic':
            has_internal_ext = field[interior].min() < field[exterior].min()

    # 4. Find amplitude of region, reject if < amp_thresh
        if cyc == 'anticyclonic':
            amp = field[interior].max() - field[exterior].mean()
        elif cyc == 'cyclonic':
            amp = field[exterior].mean() - field[interior].min()
        is_tall_eddy = amp >= amp_thresh

    # 5. Find maximum linear dimension of region, reject if < d_thresh
        #removed * has_internal_ext TS
        if np.logical_not(eddy_area_within_limits * has_internal_ext * is_tall_eddy):
        #if np.logical_not( eddy_area_within_limits * has_internal_ext):
            continue

        lon_ext = llon[exterior]
        lat_ext = llat[exterior]

        d = distance_matrix(lon_ext, lat_ext)
        #print("dmax: " ,d.max())
        is_small_eddy = d.max() < d_thresh
        
        #removed * has_internal_ext TS
        if eddy_area_within_limits * has_internal_ext * is_tall_eddy * is_small_eddy:
        #if eddy_area_within_limits * has_internal_ext * is_small_eddy:

                eddy_object_with_mass = rv_mask[tt] * region
                #make all nan = 0 but we don't have nan, only zero
                eddy_object_with_mass[np.isnan(eddy_object_with_mass)] = 0
                j_cen, i_cen = ndimage.center_of_mass(eddy_object_with_mass)
                lon_cen = np.interp(i_cen, range(0,len(lon)), lon)
                lat_cen = np.interp(j_cen, range(0,len(lat)), lat)
                lon_eddies = np.append(lon_eddies, lon_cen)
                lat_eddies = np.append(lat_eddies, lat_cen)
                # assign (and calculated) amplitude, area, and scale of eddies
                amp_eddies = np.append(amp_eddies, amp)
                #change the area because our lat grid is different (same km distance for lons and lats)
                #area = region_Npix * res**2 * len_deg_lat * len_deg_lon(lat_cen) # [km**2]
                area = region_Npix*(len_deg_lon(lat_cen)*res)**2
                area_eddies = np.append(area_eddies, area)
                scale = np.sqrt(area / np.pi) # [km]
                scale_eddies = np.append(scale_eddies, scale)
                
                #circularity
                lat_buff = np.ceil(10*scale/len_deg_lat)/10
                lon_buff = np.ceil(10*scale/len_deg_lon(lat_cen))/10

                lats_search = latsRVx[find_ind(latsRVx[:,0],lat_cen - lat_buff):find_ind(latsRVx[:,0],lat_cen+lat_buff)+1
                              ,find_ind(lonsRVx[0,:],lon_cen-lon_buff):find_ind(lonsRVx[0,:],lon_cen+lon_buff)+1]

                lons_search = lonsRVx[find_ind(latsRVx[:,0],lat_cen - lat_buff):find_ind(latsRVx[:,0],lat_cen+lat_buff)+1
                              ,find_ind(lonsRVx[0,:],lon_cen-lon_buff):find_ind(lonsRVx[0,:],lon_cen+lon_buff)+1]

                #find distances from centre
                dist_matrix = distance_sloc(lat_cen,lats_search,lon_cen,lons_search)
                #mask circle
                circle_mask = np.where(dist_matrix>scale,0,1)
                #eddy mask
                eddy_mask = np.where(regions[find_ind(latsRVx[:,0],lat_cen - lat_buff):find_ind(latsRVx[:,0],lat_cen+lat_buff)+1
                              ,find_ind(lonsRVx[0,:],lon_cen-lon_buff):find_ind(lonsRVx[0,:],lon_cen+lon_buff)+1]==iregion+1,1,0)

                #overlap
                overlap = eddy_mask*circle_mask

                circ_idx = overlap.sum()/region_Npix
                circ_eddies = np.append(circ_eddies,circ_idx)
                
                
                
                
                #removed the mask for ssh_crits
                
    return lon_eddies, lat_eddies, amp_eddies, area_eddies, scale_eddies, circ_eddies


In [80]:
def detection_plot(tt,lon,lat,eta,eta_filt,anticyc_eddies,cyc_eddies,ptype,plot_dir,findrange=True):
    """function to plot how the eddy detection alogirthm went
    
    :tt
    :lon
    :lat
    :eta
    :eta_filt
    :anticyc_eddies
    :cyc_eddies
    :ptype
    :plot_dir
    :findrange=True
    :returns: @todo
    """
    def plot_eddies():
        """@todo: Docstring for plot_eddies
        :returns: @todo
        """
        ax.plot(anticyc_eddies[0], anticyc_eddies[1], 'k^')
        ax.plot(cyc_eddies[0], cyc_eddies[1], 'kv')
    
        pass
    if ptype=='single':
        plt.close('all')
        fig=plt.figure()
        ax=fig.add_subplot(1, 1,1)

    elif ptype=='rawtoo':
        plt.close('all')
        fig=plt.figure()

        #width then height
        fig=plt.figure(figsize=(12.0,5.0))
        ax=fig.add_subplot(1, 2,1)

        #ecj range...
        #plt.contourf(lon, lat, eta_filt, levels=np.arange(-2.5,2.5,0.05))

        #cb NEMO range
        cs1=plt.contourf(lon, lat, eta_filt)
                         #levels=np.linspace(-.817,0.5,40))
        cbar=fig.colorbar(cs1,orientation='vertical')
        ax.set_title('day: ' + str(tt)+' filtered ssh')
        plot_eddies()
        
        ax=fig.add_subplot(1, 2,2)
        cs1=plt.contourf(lon, lat, eta)
                         #levels=np.linspace(-1.75,0.85,40))
        cbar=fig.colorbar(cs1,orientation='vertical')
        ax.set_title('day: ' + str(tt)+' raw ssh')
        plot_eddies()


        #determine range to plot 
        #if np.isnan(np.sum(eta_filt)):
            #plt.contourf(lon,lat, eta_filt,levels=np.linspace(np.min(np.nan_to_num(eta_filt)),np.max(np.nan_to_num(eta_filt)),50))
            #print np.min(np.nan_to_num(eta_filt))
            #print np.max(np.nan_to_num(eta_filt))
        #else:
            #plt.contourf(lon,lat, eta_filt,levels=np.linspace(np.min(eta_filt),np.max(eta_filt),50))

        #plt.clim(-0.5,0.5)
        #plt.savefig(plot_dir+'eta_filt_' + str(tt).zfill(4) + '.png', bbox_inches=0)

    pass

def eddies_list(lon_eddies_a, lat_eddies_a, amp_eddies_a, area_eddies_a, scale_eddies_a, circ_eddies_a,lon_eddies_c, lat_eddies_c, amp_eddies_c, area_eddies_c, scale_eddies_c,circ_eddies_c):
    '''
    Creates list detected eddies
    '''

    eddies = []
    
    #lon_eddies_c and lat_eddies_a are same (no of timesteps)
    for ed in range(len(lon_eddies_c)):
        eddy_tmp = {}
        eddy_tmp['lon'] = np.append(lon_eddies_a[ed], lon_eddies_c[ed])
        eddy_tmp['lat'] = np.append(lat_eddies_a[ed], lat_eddies_c[ed])
        eddy_tmp['amp'] = np.append(amp_eddies_a[ed], amp_eddies_c[ed])
        eddy_tmp['area'] = np.append(area_eddies_a[ed], area_eddies_c[ed])
        eddy_tmp['scale'] = np.append(scale_eddies_a[ed], scale_eddies_c[ed])
        eddy_tmp['circ'] = np.append(circ_eddies_a[ed], circ_eddies_c[ed])
        eddy_tmp['type'] = list(repeat('anticyclonic',len(lon_eddies_a[ed]))) + list(repeat('cyclonic',len(lon_eddies_c[ed])))
        eddy_tmp['N'] = len(eddy_tmp['lon'])
        eddies.append(eddy_tmp)

    return eddies


def eddies_init(det_eddies):
    '''
    Initializes list of eddies. The ith element of output is
    a dictionary of the ith eddy containing information about
    position and size as a function of time, as well as type.
    '''

    eddies = []

    for ed in range(det_eddies[0]['N']):
        eddy_tmp = {}
        eddy_tmp['lon'] = np.array([det_eddies[0]['lon'][ed]])
        eddy_tmp['lat'] = np.array([det_eddies[0]['lat'][ed]])
        eddy_tmp['amp'] = np.array([det_eddies[0]['amp'][ed]])
        eddy_tmp['area'] = np.array([det_eddies[0]['area'][ed]])
        eddy_tmp['scale'] = np.array([det_eddies[0]['scale'][ed]])
        eddy_tmp['type'] = det_eddies[0]['type'][ed]
        eddy_tmp['time'] = np.array([1])
        eddy_tmp['exist_at_start'] = True
        eddy_tmp['terminated'] = False
        eddies.append(eddy_tmp)

    return eddies


def load_rossrad():
    '''
    Load first baroclinic wave speed [m/s] and Rossby radius
    of deformation [km] data from rossrad.dat (Chelton et al., 1998)

    Also calculated is the first baroclinic Rossby wave speed [m/s]
    according to the formula:  cR = -beta rossby_rad**2
    '''

    #data = np.loadtxt('data/rossrad.dat')

    #cb
    data = np.loadtxt('./rossrad.dat')

    rossrad = {}
    rossrad['lat'] = data[:,0]
    rossrad['lon'] = data[:,1]
    rossrad['c1'] = data[:,2] # m/s
    rossrad['rossby_rad'] = data[:,3] # km

    R = 6371.e3 # Radius of Earth [m]
    Sigma = 2 * np.pi / (24*60*60) # Rotation frequency of Earth [rad/s]
    beta = (2*Sigma/R) * np.cos(rossrad['lat']*np.pi/180) # 1 / m s
    rossrad['cR'] = -beta * (1e3*rossrad['rossby_rad'])**2

    return rossrad


def is_in_ellipse(x0, y0, dE, d, x, y):
    '''
    Check if point (x,y) is contained in ellipse given by the equation

      (x-x1)**2     (y-y1)**2
      ---------  +  ---------  =  1
         a**2          b**2

    where:

      a = 0.5 * (dE + dW)
      b = dE
      x1 = x0 + 0.5 * (dE - dW)
      y1 = y0
    '''

    dW = np.max([d, dE])

    b = dE
    a = 0.5 * (dE + dW)

    x1 = x0 + 0.5*(dE - dW)
    y1 = y0

    return (x-x1)**2 / a**2 + (y-y1)**2 / b**2 <= 1


def len_deg_lon(lat):
    '''
    Returns the length of one degree of longitude (at latitude
    specified) in km.
    '''

    R = 6371. # Radius of Earth [km]

    return (np.pi/180.) * R * np.cos( lat * np.pi/180. )


def calculate_d(dE, lon, lat, rossrad, dt):
    '''
    Calculates length of search area to the west of central point.
    This is equal to the length of the search area to the east of
    central point (dE) unless in the tropics ( abs(lat) < 18 deg )
    in which case the distance a Rossby wave travels in one time step
    (dt, days) is used instead.
    '''

    if np.abs(lat) < 18 :
        # Rossby wave speed [km/day]
        c = interpolate.griddata(np.array([rossrad['lon'], rossrad['lat']]).T, rossrad['cR'], (lon, lat), method='linear') * 86400. / 1000.
        d = np.abs(1.75 * c * dt)
    else:
        d = dE

    return d


def track_eddies(eddies, det_eddies, tt, dt, dt_aviso, dE_aviso, rossrad, eddy_scale_min, eddy_scale_max):
    '''
    Given a map of detected eddies as a function of time (det_eddies)
    this function will update tracks of individual eddies at time step
    tt in variable eddies
    '''

    # List of unassigned eddies at time tt

    unassigned = range(det_eddies[tt]['N'])

    # For each existing eddy (t<tt) loop through unassigned eddies and assign to existing eddy if appropriate

    for ed in range(len(eddies)):

        # Check if eddy has already been terminated

        if not eddies[ed]['terminated']:

            # Define search region around centroid of existing eddy ed at last known position
    
            x0 = eddies[ed]['lon'][-1] # [deg. lon]
            y0 = eddies[ed]['lat'][-1] # [deg. lat]
            dE = dE_aviso/(dt_aviso/dt) # [km]
            d = calculate_d(dE, x0, y0, rossrad, dt) # [km]
    
            # Find all eddy centroids in search region at time tt
    
            is_near = is_in_ellipse(x0, y0, dE/len_deg_lon(y0), d/len_deg_lon(y0), det_eddies[tt]['lon'][unassigned], det_eddies[tt]['lat'][unassigned])
    
            # Check if eddies' amp  and area are between 0.25 and 2.5 of original eddy
    
            amp = eddies[ed]['amp'][-1]
            area = eddies[ed]['area'][-1]
            is_similar_amp = (det_eddies[tt]['amp'][unassigned] < amp*eddy_scale_max) * (det_eddies[tt]['amp'][unassigned] > amp*eddy_scale_min)
            is_similar_area = (det_eddies[tt]['area'][unassigned] < area*eddy_scale_max) * (det_eddies[tt]['area'][unassigned] > area*eddy_scale_min)
    
            # Check if eddies' type is the same as original eddy
    
            is_same_type = np.array([det_eddies[tt]['type'][i] == eddies[ed]['type'] for i in unassigned])
    
            # Possible eddies are those which are near, of the right amplitude, and of the same type
    
            possibles = is_near * is_similar_amp * is_similar_area * is_same_type
            if possibles.sum() > 0:
    
                # Of all found eddies, accept only the nearest one
    
                dist = np.sqrt((x0-det_eddies[tt]['lon'][unassigned])**2 + (y0-det_eddies[tt]['lat'][unassigned])**2)
                nearest = dist == dist[possibles].min()
                next_eddy = unassigned[np.where(nearest * possibles)[0][0]]
    
                # Add coordinatse and properties of accepted eddy to trajectory of eddy ed
    
                eddies[ed]['lon'] = np.append(eddies[ed]['lon'], det_eddies[tt]['lon'][next_eddy])
                eddies[ed]['lat'] = np.append(eddies[ed]['lat'], det_eddies[tt]['lat'][next_eddy])
                eddies[ed]['amp'] = np.append(eddies[ed]['amp'], det_eddies[tt]['amp'][next_eddy])
                eddies[ed]['area'] = np.append(eddies[ed]['area'], det_eddies[tt]['area'][next_eddy])
                eddies[ed]['scale'] = np.append(eddies[ed]['scale'], det_eddies[tt]['scale'][next_eddy])
                eddies[ed]['time'] = np.append(eddies[ed]['time'], tt+1)
    
                # Remove detected eddy from list of eddies available for assigment to existing trajectories
    
                unassigned.remove(next_eddy)

            # Terminate eddy otherwise

            else:

                eddies[ed]['terminated'] = True

    # Create "new eddies" from list of eddies not assigned to existing trajectories

    if len(unassigned) > 0:

        for un in unassigned:

            eddy_tmp = {}
            eddy_tmp['lon'] = np.array([det_eddies[tt]['lon'][un]])
            eddy_tmp['lat'] = np.array([det_eddies[tt]['lat'][un]])
            eddy_tmp['amp'] = np.array([det_eddies[tt]['amp'][un]])
            eddy_tmp['area'] = np.array([det_eddies[tt]['area'][un]])
            eddy_tmp['scale'] = np.array([det_eddies[tt]['scale'][un]])
            eddy_tmp['type'] = det_eddies[tt]['type'][un]
            eddy_tmp['time'] = np.array([tt+1])
            eddy_tmp['exist_at_start'] = False
            eddy_tmp['terminated'] = False
            eddies.append(eddy_tmp)

    return eddies



In [65]:
latsRVx = latsRVx.values
lonsRVx = lonsRVx.values

## Eddy Detection Algorithm

Adjust parameters in params.py: T, res, dt, pathroot

In [79]:
%%time
## Eddy Detection Algorithm

'''

  Software for the tracking of eddies in
  OFAM model output following Chelton et
  al., Progress in Oceanography, 2011.

'''

# Load required modules

import numpy as np

import matplotlib
# Turn the followin on if you are running on storm sometimes - Forces matplotlib to not use any Xwindows backend.
matplotlib.use('Agg')


from matplotlib import pyplot as plt
#import eddy_functions as eddy

# Load parameters

from params import *
#TS - overwrite params T

T = 365
# Load latitude and longitude vectors and restrict to domain of interest

# lon, lat = eddy.load_lonlat(run)
lon, lat = load_lonlat(run)
##chris' dodgy hack for not having the eric find_nearest function...
#i1=0
#i2=2000
#j1=0
#j2=2000
# lon, lat, i1, i2, j1, j2 = eddy.restrict_lonlat(lon, lat, lon1, lon2, lat1, lat2)
lon, lat, i1, i2, j1, j2 = restrict_lonlat(lon, lat, lon1, lon2, lat1, lat2)

# Loop over time


lon_eddies_a = []
lat_eddies_a = []
amp_eddies_a = []
area_eddies_a = []
scale_eddies_a = []
circ_eddies_a = []
lon_eddies_c = []
lat_eddies_c = []
amp_eddies_c = []
area_eddies_c = []
scale_eddies_c = []
circ_eddies_c= []

print ('eddy detection started')
print ("number of time steps to loop over: ",T)
for tt in range(T):
    print ("timestep: ",tt+1,". out of: ", T)

    # Load map of sea surface height (SSH)
 
    #eta, eta_miss = eddy.load_eta(run, tt, i1, i2, j1, j2)
    eta, eta_miss = load_eta(run, tt, i1, i2, j1, j2)
    #eta = eddy.remove_missing(eta, missing=eta_miss, replacement=np.nan)
    
    #changed eta in that code so should be fine
    eta = remove_missing(eta, missing=eta_miss, replacement=np.nan)
    #eddy.quick_plot(eta,findrange=True)
    # 
    ## Spatially filter SSH field
    # 
    #eta_filt = eddy.spatial_filter(eta, lon, lat, res, cut_lon, cut_lat)
    eta_filt = spatial_filter(eta, lon, lat, res, cut_lon, cut_lat)
    #eddy.quick_plot(eta_filt,findrange=True)
    # 
    ## Detect lon and lat coordinates of eddies
    #
    #lon_eddies, lat_eddies, amp, area, scale = eddy.detect_eddies(eta_filt, lon, lat, ssh_crits, res, Npix_min, Npix_max, amp_thresh, d_thresh, cyc='anticyclonic')
    
    #removed ssh_crits and added rv_mask
    lon_eddies, lat_eddies, amp, area, scale, circ = detect_eddies(rv_mask[tt], eta_filt, lon, lat, res, Npix_min, Npix_max, amp_thresh, d_thresh, cyc='anticyclonic')
    lon_eddies_a.append(lon_eddies)
    lat_eddies_a.append(lat_eddies)
    amp_eddies_a.append(amp)
    area_eddies_a.append(area)
    scale_eddies_a.append(scale)
    circ_eddies_a.append(circ)

    #lon_eddies, lat_eddies, amp, area, scale = eddy.detect_eddies(eta_filt, lon, lat, ssh_crits, res, Npix_min, Npix_max, amp_thresh, d_thresh, cyc='cyclonic')
    lon_eddies, lat_eddies, amp, area, scale, circ = detect_eddies(rv_mask[tt], eta_filt, lon, lat,  res, Npix_min, Npix_max, amp_thresh, d_thresh, cyc='cyclonic')
    lon_eddies_c.append(lon_eddies)
    lat_eddies_c.append(lat_eddies)
    amp_eddies_c.append(amp)
    area_eddies_c.append(area)
    scale_eddies_c.append(scale)
    circ_eddies_c.append(circ)
 
    # Plot map of filtered SSH field

    eddies_a=(lon_eddies_a[tt],lat_eddies_a[tt])
    eddies_c=(lon_eddies_c[tt],lat_eddies_c[tt])
    #eddy.detection_plot(tt,lon,lat,eta,eta_filt,eddies_a,eddies_c,'rawtoo',plot_dir,findrange=False)
    detection_plot(tt,lon,lat,eta,eta_filt,eddies_a,eddies_c,'rawtoo',plot_dir,findrange=False)

# Combine eddy information from all days into a list

#eddies = eddy.eddies_list(lon_eddies_a, lat_eddies_a, amp_eddies_a, area_eddies_a, scale_eddies_a, lon_eddies_c, lat_eddies_c, amp_eddies_c, area_eddies_c, scale_eddies_c)
eddies = eddies_list(lon_eddies_a, lat_eddies_a, amp_eddies_a, area_eddies_a, scale_eddies_a,circ_eddies_a, lon_eddies_c, lat_eddies_c, amp_eddies_c, area_eddies_c, scale_eddies_c, circ_eddies_c)

#turn on/off save
np.savez(data_dir+'eddy_det_BP12-CNCLNG01_2005_2009new', eddies=eddies)

eddy detection started
number of time steps to loop over:  365
timestep:  1 . out of:  365
timestep:  2 . out of:  365
timestep:  3 . out of:  365
timestep:  4 . out of:  365
timestep:  5 . out of:  365
timestep:  6 . out of:  365
timestep:  7 . out of:  365
timestep:  8 . out of:  365
timestep:  9 . out of:  365
timestep:  10 . out of:  365
timestep:  11 . out of:  365
timestep:  12 . out of:  365
timestep:  13 . out of:  365
timestep:  14 . out of:  365
timestep:  15 . out of:  365
timestep:  16 . out of:  365
timestep:  17 . out of:  365
timestep:  18 . out of:  365
timestep:  19 . out of:  365
timestep:  20 . out of:  365
timestep:  21 . out of:  365
timestep:  22 . out of:  365
timestep:  23 . out of:  365
timestep:  24 . out of:  365
timestep:  25 . out of:  365
timestep:  26 . out of:  365
timestep:  27 . out of:  365
timestep:  28 . out of:  365
timestep:  29 . out of:  365
timestep:  30 . out of:  365
timestep:  31 . out of:  365
timestep:  32 . out of:  365
timestep:  33 . ou

timestep:  276 . out of:  365
timestep:  277 . out of:  365
timestep:  278 . out of:  365
timestep:  279 . out of:  365
timestep:  280 . out of:  365
timestep:  281 . out of:  365
timestep:  282 . out of:  365
timestep:  283 . out of:  365
timestep:  284 . out of:  365
timestep:  285 . out of:  365
timestep:  286 . out of:  365
timestep:  287 . out of:  365
timestep:  288 . out of:  365
timestep:  289 . out of:  365
timestep:  290 . out of:  365
timestep:  291 . out of:  365
timestep:  292 . out of:  365
timestep:  293 . out of:  365
timestep:  294 . out of:  365
timestep:  295 . out of:  365
timestep:  296 . out of:  365
timestep:  297 . out of:  365
timestep:  298 . out of:  365
timestep:  299 . out of:  365
timestep:  300 . out of:  365
timestep:  301 . out of:  365
timestep:  302 . out of:  365
timestep:  303 . out of:  365
timestep:  304 . out of:  365
timestep:  305 . out of:  365
timestep:  306 . out of:  365
timestep:  307 . out of:  365
timestep:  308 . out of:  365
timestep: 

TypeError: eddies_list() takes 10 positional arguments but 12 were given

In [86]:
eddies = eddies_list(lon_eddies_a, lat_eddies_a, amp_eddies_a, area_eddies_a, scale_eddies_a,circ_eddies_a, lon_eddies_c, lat_eddies_c, amp_eddies_c, area_eddies_c, scale_eddies_c, circ_eddies_c)


In [88]:
np.savez('/home/tsmith/scratch/Okubo/BP12-CNCLNG01/depth/Eddy_Detect/Eddy_detect_BP12-CNCLNG01_2005_2009new', eddies=eddies)


In [ ]:
#import the eddy array

In [89]:
eddy_det = np.load('/home/tsmith/scratch/Okubo/BP12-CNCLNG01/depth/Eddy_Detect/Eddy_detect_BP12-CNCLNG01_2005_2009new.npz',allow_pickle=True)

eddy_array = eddy_det['eddies']


In [91]:
len(eddy_array)

365

In [92]:
eddy_sum = 0
for i in range(len(eddy_array)):
    num = eddy_array[i]['N']
    eddy_sum += num

eddy_sum

26150

### Find Big Eddy

In [95]:
#list of max scale
max_scale = []
for i in range(len(eddy_array)):
        max_scale += [np.max(eddy_array[i]['scale'])]
       

In [96]:
len(max_scale)

365

In [97]:
t = max_scale.index(np.max(max_scale))

In [98]:
t

5

In [99]:
max_scale[t]

113.35060496437889

In [100]:
ed = find_ind(eddy_array[t]['scale'],max_scale[t])

In [101]:
ed

33